In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('./utils')
sys.path.append('./utils/APIs')

import matplotlib.pyplot as plt
import torch
import timm
import argparse
from Config import config
from utils.common import read_from_file, save_model, write_to_file, train_val_split
from utils.DataProcess import Processor
from Trainer import Trainer
from PreTrainer import PreTrainer
from Models.mamba.models_insect import build_vssm_models_ as mamba_model
from Models.OTEModel import Model

In [2]:
# args
# 训练模型
do_train = True
# 预测测试集数据
do_test = True
# 已经训练好的模型路径
load_model_path = './save_models/CMAT/pytorch_model.bin'
# 设置学习率
lr = 3e-6
# 设置权重衰减
weight_decay = 1e-2
# 设置训练轮数
epoch = 20
# 文本分析模型
text_pretrained_model = 'huawei-noah/TinyBERT_General_4L_312D'
cv_pretrained_model = 'WAVM'
# 融合模型类别
fuse_model_type = 'CMAT'
# 仅用文本预测
text_only = False
# 仅用图像预测
img_only = False

config.learning_rate = lr
config.weight_decay = weight_decay
config.epoch = epoch
config.bert_name = text_pretrained_model
config.resnet_name = cv_pretrained_model
config.fuse_model_type = fuse_model_type
config.load_model_path = load_model_path
config.only = 'img' if img_only else None
config.only = 'text' if text_only else None
if img_only and text_only: config.only = None
print('TextModel: {}, ImageModel: {}, FuseModel: {}'.format(config.bert_name,config.resnet_name, config.fuse_model_type))

TextModel: huawei-noah/TinyBERT_General_4L_312D, ImageModel: WAVM, FuseModel: CMAT


In [3]:
# Initilaztion
processor = Processor(config)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# PreTrain
dataset_file_path = '../dataset/CornDataset/csv/train/train_data.csv'
data = read_from_file(dataset_file_path)
train_data, val_data = train_val_split(data)
train_loader = processor(train_data, config.train_params, 'train')
val_loader = processor(val_data, config.val_params, 'train')
# preModel = Model(config)
# preTrainer = PreTrainer(config, processor, preModel, device)
# best_lr = preTrainer.train_to_get_best_lr(train_loader)
# config.learning_rate = best_lr
config.learning_rate = 8.05755185e-06

----- [Loading]


In [ ]:
model = Model(config)
trainer = Trainer(config, processor, model, device)

In [ ]:
plt_train_loss = []
plt_val_loss = []

In [ ]:
# Train
def train():
    best_acc = 0
    epoch = config.epoch
    for e in range(epoch):
        print('-' * 20 + ' ' + 'Epoch ' + str(e+1) + ' ' + '-' * 20)
        tloss = trainer.train(train_loader)
        plt_train_loss.append(tloss)
        print('Train Loss: {}'.format(tloss))
        vloss, vacc = trainer.valid(val_loader)
        plt_val_loss.append(vloss)
        print('Valid Loss: {}'.format(vloss))
        print('Valid Acc: {}'.format(vacc))
        if vacc > best_acc:
            best_acc = vacc
            save_path = './save_models'
            save_model(save_path, config.fuse_model_type, model)
            print('Update best model!')
        print()
    # plt
    train_loss_y = plt_train_loss
    train_loss_x = range(len(plt_train_loss))

    val_loss_y = plt_val_loss
    val_loss_x = range(len(plt_val_loss))

    plt.figure()

    # 去除顶部和右边框框
    ax = plt.axes()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.xlabel('iters')    # x轴标签
    plt.ylabel('loss')     # y轴标签

    # 以x_train_loss为横坐标，y_train_loss为纵坐标，曲线宽度为1，实线，增加标签，训练损失，
    # 默认颜色，如果想更改颜色，可以增加参数color='red',这是红色。
    plt.plot(train_loss_x, train_loss_y, linewidth=1, linestyle="solid", label="train loss")
    plt.plot(val_loss_x, val_loss_y, linewidth=1, linestyle="solid", label="val loss")
    plt.legend()
    plt.title('Loss curve')
    plt.show()

In [ ]:
# Test
def test():
    test_file_path = '../dataset/CornDataset/csv/test/test_data.csv'
    test_data = read_from_file(test_file_path)
    test_loader = processor(test_data, config.test_params,'test')

    # if config.load_model_path is not None:
    #     model.load_state_dict(torch.load(config.load_model_path))

    outputs = trainer.predict(test_loader)
    write_to_file(test_file_path, outputs)

In [ ]:
# main
if __name__ == "__main__":
    if do_train:
        train()
    
    if do_test:
        if load_model_path is None and not do_train:
            print('请输入已训练好模型的路径load_model_path或者选择添加do_train arg')
        else:
            test()